In [1]:
# ============================================================
# CELL 1: System Dependencies
# ============================================================
!apt-get update -qq
!apt-get install -y -qq \
    libcairo2-dev libpango1.0-dev libfreetype6-dev \
    libharfbuzz-dev ffmpeg pkg-config python3-dev \
    texlive-latex-base texlive-latex-extra \
    texlive-fonts-recommended dvipng

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [4]:
# ============================================================
# CELL 2: Python Dependencies
# ============================================================
!pip install -q manim groq gtts mutagen langchain-groq
print("✅ All dependencies installed")

✅ All dependencies installed


In [5]:
# ============================================================
# CELL 2: Imports & Directory Setup
# ============================================================
import os
import json
import re
import uuid
import random
import asyncio
import subprocess
from typing import List, Dict, Optional
from pathlib import Path
from dotenv import load_dotenv
from IPython.display import Audio, Video, display

# Create directory structure
for d in ['src/core','src/utils','src/config',
          'data/context_learning','data/rag/chroma_db',
          'data/rag/manim_docs','output']:
    os.makedirs(d, exist_ok=True)

print('✅ Imports & directories ready')

✅ Imports & directories ready


In [33]:
# ============================================================
# CELL 3: API Configuration
# ============================================================
import os
from groq import Groq

GROQ_API_KEY = "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"
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
client = Groq(api_key=GROQ_API_KEY)
MODEL  = "llama-3.3-70b-versatile"
print("✅ Groq configured")

✅ Groq configured


In [34]:
# ============================================================
# CELL 4: Create Mock Modules
# ============================================================

# --- config ---
with open('src/config/__init__.py','w') as f: f.write('')
with open('src/config/config.py','w') as f:
    f.write('''
class Config:
    OUTPUT_DIR = "output"
    CHROMA_DB_PATH = "data/rag/chroma_db"
    MANIM_DOCS_PATH = "data/rag/manim_docs"
    CONTEXT_LEARNING_PATH = "data/context_learning"
    EMBEDDING_MODEL = "azure/text-embedding-3-large"
''')

# --- utils ---
with open('src/utils/__init__.py','w') as f: f.write('')
with open('src/utils/utils.py','w') as f:
    f.write('''
def _print_response(response, verbose=False):
    if verbose: print(response)

def _extract_code(text, language="python"):
    import re
    matches = re.findall(f"```{language}(.*?)```", text, re.DOTALL)
    return matches[0].strip() if matches else text

def extract_xml(text, tag="content"):
    import re
    match = re.search(f"<{tag}>(.*?)</{tag}>", text, re.DOTALL)
    return match.group(1).strip() if match else text
''')

# allowed models
with open('src/utils/allowed_models.json','w') as f:
    json.dump({"allowed_models": ["gemini/gemini-1.5-pro-002","gpt-4o","llama-3.3-70b-versatile"]}, f)

# --- mllm_tools ---
os.makedirs('mllm_tools', exist_ok=True)
with open('mllm_tools/__init__.py','w') as f: f.write('')
with open('mllm_tools/utils.py','w') as f:
    f.write('def _prepare_text_inputs(text): return text\n')
with open('mllm_tools/litellm.py','w') as f:
    f.write('''
class LiteLLMWrapper:
    def __init__(self, model_name, temperature=0.7, print_cost=True, verbose=False, use_langfuse=False):
        self.model_name = model_name
    def generate_text(self, prompt, **kwargs):
        return f"Mock: {prompt[:40]}..."
''')

# --- video_planner ---
with open('src/core/__init__.py','w') as f: f.write('')
with open('src/core/video_planner.py','w') as f:
    f.write('''
class VideoPlanner:
    def __init__(self, planner_model, helper_model=None, output_dir="output",
                 print_response=False, use_context_learning=False,
                 context_learning_path="", use_rag=False, session_id="",
                 chroma_db_path="", manim_docs_path="", embedding_model="",
                 use_langfuse=False):
        self.planner_model = planner_model
        self.output_dir = output_dir
        self.use_rag = False

    def generate_scene_outline(self, topic, description, session_id):
        return f"""<content>
<SCENE_1>Introduction to {topic}: {description}</SCENE_1>
<SCENE_2>Core concepts and visual demonstration</SCENE_2>
<SCENE_3>Practical examples and applications</SCENE_3>
<SCENE_4>Summary and conclusion</SCENE_4>
</content>"""

    async def _generate_scene_implementation_single(self, topic, description, scene_outline_i, i, file_prefix, session_id, scene_trace_id):
        return f"Detailed implementation for {topic} scene {i}: {str(scene_outline_i).strip()}"

    async def generate_scene_implementation(self, topic, description, plan, session_id):
        return [f"Plan for scene {i}" for i in range(1,5)]

    async def generate_scene_implementation_concurrently(self, topic, description, plan, session_id, semaphore):
        return await self.generate_scene_implementation(topic, description, plan, session_id)
''')

# --- code_generator (now uses Groq) ---
with open('src/core/code_generator.py','w') as f:
    f.write('''
import os

class CodeGenerator:
    def __init__(self, scene_model, helper_model=None, output_dir="output",
                 print_response=False, use_rag=False, use_context_learning=False,
                 context_learning_path="", chroma_db_path="", manim_docs_path="",
                 embedding_model="", use_visual_fix_code=False, use_langfuse=False,
                 session_id="", groq_client=None, groq_model="llama-3.3-70b-versatile"):
        self.output_dir = output_dir
        self.groq_client = groq_client
        self.groq_model  = groq_model

    def _call_groq(self, topic, duration, extra=""):
        if not self.groq_client:
            return self._mock_code(topic, 1)
        prompt = f"""Generate ONLY raw Python Manim CE code. No markdown, no backticks.
Class name: GeneratedVideo(Scene)
Topic: {topic}
Duration: ~{duration:.0f}s
Rules:
- from manim import *
- import numpy as np
- Title .to_edge(UP), diagram at ORIGIN, formula .to_edge(DOWN)
- FadeOut(*self.mobjects) between stages
- Animate clearly step by step
{extra}"""
        resp = self.groq_client.chat.completions.create(
            model=self.groq_model,
            messages=[{"role":"user","content":prompt}]
        )
        code = resp.choices[0].message.content.strip()
        if "```" in code:
            code = code.split("```python")[-1].split("```")[0].strip()
        return code

    def _mock_code(self, topic, scene_number):
        return f"""
from manim import *
import numpy as np

class GeneratedVideo(Scene):
    def construct(self):
        title = Text("{topic}", font_size=48).to_edge(UP)
        self.play(Write(title))
        self.wait(1)
        body = Text("Scene {scene_number}", font_size=32).move_to(ORIGIN)
        self.play(FadeIn(body))
        self.wait(2)
        self.play(FadeOut(*self.mobjects))
"""

    def generate_manim_code(self, topic, description, scene_outline,
                            scene_implementation, scene_number,
                            additional_context=None, scene_trace_id="",
                            session_id="", rag_queries_cache=None):
        code = self._call_groq(topic, 30)
        log  = f"Generated code for {topic} scene {scene_number}"
        return code, log

    def fix_code_errors(self, implementation_plan, code, error,
                        scene_trace_id, topic, scene_number,
                        session_id, rag_queries_cache):
        extra = f"Fix this error:\n{error[:500]}"
        fixed = self._call_groq(topic, 30, extra)
        log   = f"Fixed error for scene {scene_number}: {error[:100]}"
        return fixed, log

    def visual_self_reflection(self, *args, **kwargs):
        return "Reflection done", False
''')

# --- video_renderer (runs real Manim) ---
with open('src/core/video_renderer.py','w') as f:
    f.write('''
import os, asyncio, subprocess, re

class VideoRenderer:
    def __init__(self, output_dir="output", print_response=False, use_visual_fix_code=False):
        self.output_dir = output_dir

    async def render_scene(self, code, file_prefix, curr_scene, curr_version,
                           code_dir, media_dir, max_retries=3,
                           use_visual_fix_code=False, visual_self_reflection_func=None,
                           banned_reasonings=None, scene_trace_id="",
                           topic="", session_id=""):
        py_path = os.path.join(code_dir, f"{file_prefix}_scene{curr_scene}_v{curr_version}.py")
        with open(py_path, "w") as f:
            f.write(code)

        result = subprocess.run(
            ["python", "-m", "manim", py_path, "GeneratedVideo",
             "-ql", "--media_dir", media_dir, "--disable_caching"],
            capture_output=True, text=True, timeout=120
        )

        if result.returncode == 0:
            scene_dir = os.path.join(self.output_dir, file_prefix, f"scene{curr_scene}")
            os.makedirs(scene_dir, exist_ok=True)
            with open(os.path.join(scene_dir, "succ_rendered.txt"), "w") as f:
                f.write("success")
            return code, None
        else:
            error = result.stderr[-1000:] if result.stderr else "Unknown error"
            print(f"Render error (scene {curr_scene}): {error[-300:]}")
            return code, error

    def combine_videos(self, topic):
        file_prefix = re.sub(r"[^a-z0-9_]+", "_", topic.lower())
        topic_dir   = os.path.join(self.output_dir, file_prefix)
        combined    = os.path.join(topic_dir, f"{file_prefix}_combined.mp4")
        os.makedirs(topic_dir, exist_ok=True)
        # Find rendered mp4 files
        import glob
        vids = sorted(glob.glob(os.path.join(self.output_dir, file_prefix, "**/*.mp4"), recursive=True))
        if not vids:
            print(f"No videos found to combine for {topic}")
            return
        with open("/tmp/concat.txt", "w") as f:
            for v in vids:
                f.write(f"file \'{v}\'\n")
        subprocess.run(["ffmpeg","-y","-f","concat","-safe","0","-i","/tmp/concat.txt","-c","copy",combined], capture_output=True)
        print(f"Combined video: {combined}")
''')

# --- parse_video ---
with open('src/core/parse_video.py','w') as f:
    f.write('''
def get_images_from_video(video_path): return []
def image_with_most_non_black_space(images): return images[0] if images else ""
''')

# --- task_generator ---
os.makedirs('task_generator', exist_ok=True)
with open('task_generator/__init__.py','w') as f:
    f.write('def get_banned_reasonings(): return []\n')
with open('task_generator/prompts_raw.py','w') as f:
    f.write('''
_code_font_size = "Use readable font sizes"
_code_disable   = "Disable unnecessary features"
_code_limit     = "Keep code concise"
_prompt_manim_cheatsheet = "Use Scene, self.play(), self.wait(), Text(), MathTex(), FadeIn/Out()"
''')

print('✅ All mock modules created')

✅ All mock modules created


In [35]:
with open('src/core/code_generator.py', 'r') as f:
    content = f.read()

content = content.replace(
    'extra = f"Fix this error: \n{error[:500]}"',
    'extra = f"Fix this error: {error[:500]}"'
)

with open('src/core/code_generator.py', 'w') as f:
    f.write(content)

print("✅ Fixed!")
with open('src/core/code_generator.py', 'r') as f:
    lines = f.readlines()

# Fix lines 63-64 (index 63-64 = lines 64-65)
if 'extra = f"Fix this error:' in lines[63]:
    lines[63] = f'        extra = f"Fix this error: {{error[:500]}}"\n'
    lines[64] = ''  # remove the orphaned second line

with open('src/core/code_generator.py', 'w') as f:
    f.writelines(lines)

print("✅ Patched! Now re-run Cell 5.")
with open('src/core/video_renderer.py', 'r') as f:
    lines = f.readlines()

print(lines[44:50])  # show lines 45-50 so we can see the broken f-string

✅ Fixed!
✅ Patched! Now re-run Cell 5.
['        with open("/tmp/concat.txt", "w") as f:\n', '            for v in vids:\n', '                f.write(f"file \'{v}\'\n', '")\n', '        subprocess.run(["ffmpeg","-y","-f","concat","-safe","0","-i","/tmp/concat.txt","-c","copy",combined], capture_output=True)\n', '        print(f"Combined video: {combined}")\n']


In [36]:
with open('src/core/video_renderer.py', 'r') as f:
    lines = f.readlines()

# Lines 47-48 (index 46-47) are a split f-string
if lines[46].strip().startswith("f.write(f\"file "):
    lines[46] = "                f.write(f\"file '{v}'\\n\")\n"
    lines[47] = ''  # remove the orphaned closing quote line

with open('src/core/video_renderer.py', 'w') as f:
    f.writelines(lines)

print("✅ Patched video_renderer.py! Now re-run Cell 5.")

✅ Patched video_renderer.py! Now re-run Cell 5.


In [37]:
# ============================================================
# CELL 5: Import All Modules
# ============================================================
load_dotenv(override=True)

from mllm_tools.litellm import LiteLLMWrapper
from mllm_tools.utils import _prepare_text_inputs
from src.core.video_planner import VideoPlanner
from src.core.code_generator import CodeGenerator
from src.core.video_renderer import VideoRenderer
from src.utils.utils import _print_response, _extract_code, extract_xml
from src.config.config import Config
from src.core.parse_video import get_images_from_video, image_with_most_non_black_space
from task_generator import get_banned_reasonings
from task_generator.prompts_raw import _code_font_size, _code_disable, _code_limit, _prompt_manim_cheatsheet

with open('src/utils/allowed_models.json') as f:
    allowed_models = json.load(f).get('allowed_models', [])

print('✅ All modules imported')

✅ All modules imported


In [51]:
# ============================================================
# CELL 6: Topic Profiles (Visual Style + Formulas + Graphs)
# ============================================================

TOPIC_PROFILES = {
    'math': {
        'keywords': ['calculus','derivative','integral','limit','matrix','vector',
                     'algebra','geometry','trigonometry','fourier','eigenvalue',
                     'polynomial','function','math','mathematics','pythagorean'],
        'formulas': [
            r'\frac{d}{dx}[f(g(x))] = f\'(g(x)) \cdot g\'(x)',
            r'\int_a^b f(x)\,dx = F(b) - F(a)',
            r'e^{i\pi} + 1 = 0',
            r'a^2 + b^2 = c^2',
            r'\sum_{n=0}^{\infty} \frac{x^n}{n!} = e^x',
        ],
        'graph_instructions': '''
WHEN TO USE WHAT:
- Any function (sin, cos, exp, log, polynomial) → Axes() + ParametricFunction()
- Derivative → tangent Line() moving along curve on Axes()
- Integral → shaded area between curve and x-axis using Axes()
- Matrix → Rectangle() grid cells with Text() numbers inside
- Vectors → Arrow() from ORIGIN on Axes()
- Geometry (triangle, circle) → Polygon(), Circle() centered at ORIGIN
- Taylor series → plot exact curve then add polynomial approximations one by one
''',
        'visual_instructions': '''
- Axes() for ALL function plots, x_length=7, y_length=4, move_to(ORIGIN)
- MathTex(r"...") for all formulas at .to_edge(DOWN)
- Create(graph) to animate curve drawing (not FadeIn)
- Brace() to annotate measurements
- Color formula terms: BLUE=variable, YELLOW=constant, GREEN=result
- Text title .to_edge(UP), diagram .move_to(ORIGIN), formula .to_edge(DOWN)
'''
    },

    'physics': {
        'keywords': ['physics','mechanics','quantum','force','energy','momentum',
                     'velocity','gravity','newton','electric','magnetic','wave',
                     'pendulum','projectile','circuit','thermodynamics'],
        'formulas': [
            r'F = ma', r'E = mc^2', r'KE = \frac{1}{2}mv^2',
            r'V = IR', r'p = mv', r'T = 2\pi\sqrt{\frac{L}{g}}'
        ],
        'graph_instructions': '''
WHEN TO USE WHAT:
- Force problems → Arrow() objects on a central Dot() or Rectangle() object
- Projectile motion → ParametricFunction() parabola on Axes()
- Wave/oscillation → ParametricFunction(lambda t: [t, sin(t), 0]) on Axes()
- Pendulum → Line() rotating about fixed top Dot(), bob as Circle() at end
- Electric field → multiple Arrow() objects radiating from charge Dot()
- Circuit → Rectangle() for resistor, Circle() for battery, Line() for wires
- Energy bar chart → Rectangle() bars for KE and PE changing height over time
- SHM → sine wave on Axes() with amplitude and period labeled with Brace()
''',
        'visual_instructions': '''
- Arrow() for all forces, labeled with MathTex(r"F=ma")
- Axes() for motion graphs (position/velocity/time)
- Color code: RED=force, BLUE=velocity, GREEN=acceleration, YELLOW=energy
- Each formula term in unique color using .set_color()
- Text title .to_edge(UP), diagram .move_to(ORIGIN), formula .to_edge(DOWN)
'''
    },

    'dsa': {
        'keywords': ['array','tree','graph','stack','queue','sorting','bfs','dfs',
                     'recursion','algorithm','complexity','binary','linked list',
                     'bubble sort','merge sort','quick sort','heap','hash','dsa'],
        'formulas': [
            r'O(1),\ O(\log n),\ O(n),\ O(n \log n),\ O(n^2)',
            r'T(n) = 2T(n/2) + O(n)',
            r'O(V+E)', r'O(n^2)'
        ],
        'graph_instructions': '''
WHEN TO USE WHAT:
- Array/list → VGroup of Square() cells arranged RIGHT, Text() number inside each
- Sorting (bubble/merge/quick) → Square() array, animate swaps by changing fill_color
  YELLOW=comparing, RED=swapping, GREEN=sorted
- Linked list → Rectangle() nodes connected by Arrow() pointing right
- Binary tree / BST → Dot() nodes, Line() edges, root at top center, children below
- Graph (BFS/DFS) → Dot() nodes at positions, Line() edges, color visited nodes GREEN
- Stack → Rectangle() blocks stacking vertically UP, label PUSH/POP with Text()
- Queue → Rectangle() blocks in a row, ENQUEUE right, DEQUEUE left with Arrow()
- Hash table → VGroup of Rectangle() buckets with Text() keys inside
- Recursion tree → Dot() nodes branching downward, Text() labels showing subproblems
- Big-O comparison → Axes() with multiple ParametricFunction() curves, one per complexity
''',
        'visual_instructions': '''
- Square(side_length=0.8) for array cells, arrange(RIGHT, buff=0.1), move_to(ORIGIN)
- SurroundingRectangle(cell, color=YELLOW) for current active element
- Dot(radius=0.3) for graph/tree nodes, Line() for edges
- Color: YELLOW=active/current, GREEN=done/sorted, RED=swap/error, BLUE=default
- MathTex(r"O(n^2)") complexity shown at end .to_edge(DOWN)
- Text title .to_edge(UP), diagram .move_to(ORIGIN), formula .to_edge(DOWN)
'''
    },

    'ml': {
        'keywords': ['machine learning','regression','classification','gradient descent',
                     'neural network','training','loss','model','ml','ai','knn',
                     'logistic','linear','decision tree','random forest','svm',
                     'clustering','k-means','naive bayes'],
        'formulas': [
            r'J(\theta) = \frac{1}{2m}\sum(h_\theta(x)-y)^2',
            r'\theta := \theta - \alpha \nabla J',
            r'h_\theta(x) = \frac{1}{1+e^{-\theta^T x}}',
            r'F_1 = \frac{2 \cdot P \cdot R}{P + R}',
        ],
        'graph_instructions': '''
WHEN TO USE WHAT:
- Linear regression → Axes() with Dot() data points + Line() best fit
- Logistic regression → Axes() with sigmoid curve ParametricFunction(lambda t: [t, 1/(1+exp(-t)), 0])
  + vertical DashedLine() at decision boundary x=0, color LEFT=class0, RIGHT=class1
- Decision boundary → Axes() with Dot() points colored by class + Line() separator
- Gradient descent → Axes() with parabola loss curve + Dot() ball rolling down to minimum
- Loss/training curve → Axes() with decreasing ParametricFunction() curve (epoch vs loss)
- KNN → Axes() with colored Dot() points + Circle() around query point showing k neighbors
- K-Means → Axes() with Dot() clusters, animate centroids moving with Arrow()
- Neural network → Circle() neurons in columns (BLUE=input, GREEN=hidden, RED=output)
  connected by Line() weights, animate forward pass left to right
- Decision tree → Rectangle() nodes with Text() conditions, Arrow() branches YES/NO
- Confusion matrix → 2x2 VGroup of Rectangle() cells labeled TP/TN/FP/FN
- SVM → Axes() with Dot() points + Line() hyperplane + DashedLine() margins
''',
        'visual_instructions': '''
- Axes() x_length=7, y_length=4, move_to(ORIGIN) for ALL plots
- Dot() colored by class: BLUE=class0, RED=class1, GREEN=class2
- Sigmoid: ParametricFunction(lambda t: np.array([t, 1/(1+np.exp(-t)), 0]), t_range=[-4,4])
- Decision boundary: DashedLine() vertical or diagonal on Axes()
- Loss curve: decreasing smooth curve from top-left to bottom-right on Axes()
- MathTex(r"...") for all formulas at .to_edge(DOWN)
- Text title .to_edge(UP), diagram .move_to(ORIGIN), formula .to_edge(DOWN)
'''
    },

    'probability': {
        'keywords': ['probability','bayes','distribution','random','statistics',
                     'normal','gaussian','bernoulli','variance','entropy',
                     'hypothesis','p-value','confidence'],
        'formulas': [
            r'P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}',
            r'E[X] = \sum_x x \cdot P(X=x)',
            r'f(x) = \frac{1}{\sigma\sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}',
            r'Var(X) = E[X^2] - (E[X])^2',
        ],
        'graph_instructions': '''
WHEN TO USE WHAT:
- Normal distribution → Axes() + ParametricFunction() bell curve, shade ±1σ region
- Bayes theorem → two overlapping Circle() Venn diagram, label each region
- Binomial → BarChart() with bar_names showing k values
- Hypothesis test → normal curve on Axes() with RED shaded rejection region
- CDF → increasing S-curve on Axes() from 0 to 1
- Confidence interval → number line with DashedLine() bounds and center Dot()
''',
        'visual_instructions': '''
- Axes() for all distributions
- ParametricFunction() for smooth PDF curves
- BarChart() for discrete PMF
- DashedLine() for mean μ, Brace() for σ
- Shade areas with polygon or Rectangle() at low opacity
- Text title .to_edge(UP), diagram .move_to(ORIGIN), formula .to_edge(DOWN)
'''
    },

    'default': {
        'keywords': [],
        'formulas': [],
        'graph_instructions': '''
WHEN TO USE WHAT:
- Data relationships → Axes() with Dot() scatter or Line() plot
- Comparisons → BarChart(values=[..], bar_names=[..], y_range=[..], bar_width=0.5)
- Flow/steps → Rectangle() boxes connected by Arrow()
- Hierarchy → tree of Dot() nodes with Line() edges
- Circular process → Circle() nodes with CurvedArrow() between them
''',
        'visual_instructions': '''
- BLUE=primary concept, GREEN=correct/positive, RED=error/negative, YELLOW=highlight
- Animate step by step, one self.play() per concept
- Short Text() labels only, max 4 words
- Brace() to annotate sections
- Text title .to_edge(UP), diagram .move_to(ORIGIN), formula .to_edge(DOWN)
'''
    }
}

def detect_topic_profile(topic):
    t = topic.lower()
    for name, p in TOPIC_PROFILES.items():
        if name == 'default': continue
        if any(kw in t for kw in p['keywords']):
            print(f'🎨 Detected topic type: {name.upper()}')
            return p
    print('🎨 Using general visual style')
    return TOPIC_PROFILES['default']

def get_relevant_formulas(profile, max_f=4):
    fmls = profile.get('formulas', [])[:max_f]
    if not fmls: return ''
    lines = '\n'.join([f'  MathTex(r"{f}")' for f in fmls])
    return f'\nKEY FORMULAS TO SHOW:\n{lines}'

def get_graph_instructions(profile):
    return profile.get('graph_instructions', '')

print('✅ Topic profiles loaded')

✅ Topic profiles loaded


In [52]:
# ============================================================
# CELL 7: TTS + Narration
# ============================================================
from gtts import gTTS
import tempfile

def generate_narration_script(topic):
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[{
            'role': 'user',
            'content': f'Write a clear 60-second educational narration script about "{topic}". '
                       'Plain text only, no headers or bullet points. '
                       'Make it engaging and suitable for a visual explainer video.'
        }]
    )
    return resp.choices[0].message.content.strip()

def text_to_speech(script, lang='en'):
    tts = gTTS(text=script, lang=lang, slow=False)
    path = '/tmp/narration.mp3'
    tts.save(path)
    # estimate duration
    words = len(script.split())
    duration = (words / 150) * 60  # ~150 wpm
    print(f'🎙️ Audio saved: {path} (~{duration:.0f}s)')
    return path, duration

print('✅ TTS ready')

✅ TTS ready


In [53]:
# ============================================================
# CELL 8: Manim Code Generator (Groq-powered)
# ============================================================

def generate_manim_code_groq(topic, duration_seconds, visual_instructions,
                              formula_hints='', graph_instructions='', error_log=''):
    system = f'''You are a Manim CE expert. Output ONLY raw Python, no markdown.

RULES:
1. Class: GeneratedVideo(Scene)
2. Start with: from manim import *\nimport numpy as np
3. Duration: ~{duration_seconds:.0f}s total
4. LAYOUT ZONES — NO OVERLAP:
   - Title:   .to_edge(UP)     ← short title only
   - Main:    .move_to(ORIGIN) ← diagram or graph
   - Formula: .to_edge(DOWN)   ← MathTex only
5. FadeOut(*self.mobjects) between every stage
6. NO narration text on screen — audio handles it

{visual_instructions}
{graph_instructions}
{formula_hints}'''

    user = f'Topic: {topic}'
    if error_log:
        user += f'\n\nFix this error:\n{error_log[-1500:]}'

    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {'role': 'system', 'content': system},
            {'role': 'user',   'content': user}
        ]
    )
    code = resp.choices[0].message.content.strip()
    if '```' in code:
        code = code.split('```python')[-1].split('```')[0].strip()
    return code


def render_manim(code):
    """Write code to file and render with Manim. Returns (success, video_path, error)."""
    with open('main.py', 'w') as f:
        f.write(code)

    result = subprocess.run(
        ['python', '-m', 'manim', 'main.py', 'GeneratedVideo',
         '-ql', '--media_dir', 'output/media', '--disable_caching'],
        capture_output=True, text=True, timeout=180
    )

    if result.returncode == 0:
        import glob
        vids = glob.glob('output/media/videos/**/*.mp4', recursive=True)
        if vids:
            vids.sort(key=os.path.getmtime, reverse=True)
            return True, vids[0], ''
    return False, None, result.stderr


def merge_video_audio(video_path, audio_path, out='output/final_video.mp4'):
    os.makedirs('output', exist_ok=True)
    subprocess.run([
        'ffmpeg', '-y', '-i', video_path, '-i', audio_path,
        '-c:v', 'copy', '-c:a', 'aac',
        '-shortest', out
    ], capture_output=True)
    print(f'🎞️ Final video: {out}')
    return out

print('✅ Manim renderer ready')

✅ Manim renderer ready


In [54]:
# ============================================================
# CELL 9: VideoGenerator Class (Full Pipeline)
# ============================================================

class VideoGenerator:
    def __init__(self, planner_model, scene_model=None, helper_model=None,
                 output_dir='output', verbose=False, use_rag=False,
                 use_context_learning=False,
                 context_learning_path='data/context_learning',
                 chroma_db_path='data/rag/chroma_db',
                 manim_docs_path='data/rag/manim_docs',
                 embedding_model='azure/text-embedding-3-large',
                 use_visual_fix_code=False, use_langfuse=False,
                 trace_id=None, max_scene_concurrency=3,
                 groq_client=None):

        self.output_dir          = output_dir
        self.verbose             = verbose
        self.use_visual_fix_code = use_visual_fix_code
        self.groq_client         = groq_client
        self.session_id          = self._load_or_create_session_id()
        self.scene_semaphore     = asyncio.Semaphore(max_scene_concurrency)
        self.banned_reasonings   = get_banned_reasonings()

        self.planner = VideoPlanner(
            planner_model=planner_model, helper_model=helper_model,
            output_dir=output_dir, print_response=verbose,
            use_context_learning=use_context_learning,
            context_learning_path=context_learning_path,
            use_rag=use_rag, session_id=self.session_id,
            chroma_db_path=chroma_db_path, manim_docs_path=manim_docs_path,
            embedding_model=embedding_model, use_langfuse=use_langfuse
        )
        self.code_generator = CodeGenerator(
            scene_model=scene_model if scene_model else planner_model,
            helper_model=helper_model if helper_model else planner_model,
            output_dir=output_dir, print_response=verbose,
            use_rag=use_rag, use_context_learning=use_context_learning,
            context_learning_path=context_learning_path,
            chroma_db_path=chroma_db_path, manim_docs_path=manim_docs_path,
            embedding_model=embedding_model,
            use_visual_fix_code=use_visual_fix_code,
            use_langfuse=use_langfuse, session_id=self.session_id,
            groq_client=groq_client
        )
        self.video_renderer = VideoRenderer(
            output_dir=output_dir, print_response=verbose,
            use_visual_fix_code=use_visual_fix_code
        )

    def _load_or_create_session_id(self):
        f = os.path.join(self.output_dir, 'session_id.txt')
        if os.path.exists(f):
            return open(f).read().strip()
        sid = str(uuid.uuid4())
        os.makedirs(self.output_dir, exist_ok=True)
        open(f, 'w').write(sid)
        return sid

    def _save_topic_session_id(self, topic, session_id):
        fp = re.sub(r'[^a-z0-9_]+', '_', topic.lower())
        d  = os.path.join(self.output_dir, fp)
        os.makedirs(d, exist_ok=True)
        open(os.path.join(d, 'session_id.txt'), 'w').write(session_id)

    def load_implementation_plans(self, topic):
        fp   = re.sub(r'[^a-z0-9_]+', '_', topic.lower())
        path = os.path.join(self.output_dir, fp, f'{fp}_scene_outline.txt')
        if not os.path.exists(path): return {}
        outline = open(path).read()
        content = extract_xml(outline)
        n = len(re.findall(r'<SCENE_(\d+)>[^<]', content))
        plans = {}
        for i in range(1, n+1):
            p = os.path.join(self.output_dir, fp, f'scene{i}', f'{fp}_scene{i}_implementation_plan.txt')
            plans[i] = open(p).read() if os.path.exists(p) else None
            status = 'Found' if plans[i] else 'Missing'
            print(f'{status} implementation plan for scene {i}')
        return plans

    async def _generate_scene_implementation_single(self, topic, description,
                                                     scene_outline_i, i,
                                                     file_prefix, session_id, scene_trace_id):
        return await self.planner._generate_scene_implementation_single(
            topic, description, scene_outline_i, i, file_prefix, session_id, scene_trace_id
        )

    async def process_scene(self, i, scene_outline, scene_implementation,
                            topic, description, max_retries, file_prefix,
                            session_id, scene_trace_id):
        curr_scene   = i + 1
        curr_version = 0
        rag_cache    = {}
        code_dir  = os.path.join(self.output_dir, file_prefix, f'scene{curr_scene}', 'code')
        media_dir = os.path.join(self.output_dir, file_prefix, 'media')
        os.makedirs(code_dir,  exist_ok=True)
        os.makedirs(media_dir, exist_ok=True)

        async with self.scene_semaphore:
            code, log = self.code_generator.generate_manim_code(
                topic=topic, description=description,
                scene_outline=scene_outline,
                scene_implementation=scene_implementation,
                scene_number=curr_scene,
                additional_context=[_prompt_manim_cheatsheet, _code_font_size, _code_limit, _code_disable],
                scene_trace_id=scene_trace_id, session_id=session_id,
                rag_queries_cache=rag_cache
            )
            open(os.path.join(code_dir, f'{file_prefix}_scene{curr_scene}_v{curr_version}.py'), 'w').write(code)
            open(os.path.join(code_dir, f'{file_prefix}_scene{curr_scene}_v{curr_version}_log.txt'), 'w').write(log)

            while True:
                code, error = await self.video_renderer.render_scene(
                    code=code, file_prefix=file_prefix, curr_scene=curr_scene,
                    curr_version=curr_version, code_dir=code_dir, media_dir=media_dir,
                    max_retries=max_retries, use_visual_fix_code=self.use_visual_fix_code,
                    visual_self_reflection_func=self.code_generator.visual_self_reflection,
                    banned_reasonings=self.banned_reasonings,
                    scene_trace_id=scene_trace_id, topic=topic, session_id=session_id
                )
                if error is None:
                    print(f'✅ Scene {curr_scene} rendered!')
                    break
                if curr_version >= max_retries:
                    print(f'❌ Max retries reached for scene {curr_scene}')
                    break
                curr_version += 1
                code, log = self.code_generator.fix_code_errors(
                    implementation_plan=scene_implementation, code=code, error=error,
                    scene_trace_id=scene_trace_id, topic=topic,
                    scene_number=curr_scene, session_id=session_id, rag_queries_cache=rag_cache
                )
                open(os.path.join(code_dir, f'{file_prefix}_scene{curr_scene}_v{curr_version}.py'), 'w').write(code)

    def combine_videos(self, topic):
        self.video_renderer.combine_videos(topic)

    async def generate_video_pipeline(self, topic, description, max_retries=3,
                                      only_plan=False, specific_scenes=None):
        session_id = self._load_or_create_session_id()
        self._save_topic_session_id(topic, session_id)
        fp = re.sub(r'[^a-z0-9_]+', '_', topic.lower())

        print(f'🎬 Starting pipeline for: {topic}')

        # Step 1: Scene outline
        outline_path = os.path.join(self.output_dir, fp, f'{fp}_scene_outline.txt')
        if os.path.exists(outline_path):
            scene_outline = open(outline_path).read()
            print('📄 Loaded existing scene outline')
        else:
            print('📝 Generating scene outline...')
            scene_outline = self.planner.generate_scene_outline(topic, description, session_id)
            os.makedirs(os.path.join(self.output_dir, fp), exist_ok=True)
            open(outline_path, 'w').write(scene_outline)

        # Step 2: Implementation plans
        plans_dict = self.load_implementation_plans(topic)
        if not plans_dict:
            n = len(re.findall(r'<SCENE_(\d+)>[^<]', extract_xml(scene_outline)))
            plans_dict = {i: None for i in range(1, n+1)}

        missing = [k for k, v in plans_dict.items()
                   if v is None and (specific_scenes is None or k in specific_scenes)]

        if missing:
            print(f'🔧 Generating plans for scenes: {missing}')
            for sn in missing:
                m = re.search(f'<SCENE_{sn}>(.*?)</SCENE_{sn}>', extract_xml(scene_outline), re.DOTALL)
                if not m: continue
                trace_id  = str(uuid.uuid4())
                scene_dir = os.path.join(self.output_dir, fp, f'scene{sn}', 'subplans')
                os.makedirs(scene_dir, exist_ok=True)
                open(os.path.join(scene_dir, 'scene_trace_id.txt'), 'w').write(trace_id)

                plan = await self._generate_scene_implementation_single(
                    topic, description, m.group(1), sn, fp, session_id, trace_id
                )
                plans_dict[sn] = plan
                plan_path = os.path.join(self.output_dir, fp, f'scene{sn}', f'{fp}_scene{sn}_implementation_plan.txt')
                os.makedirs(os.path.dirname(plan_path), exist_ok=True)
                open(plan_path, 'w').write(plan)
                print(f'✅ Saved plan for scene {sn}')

        if only_plan:
            print('📋 only_plan=True — done.')
            return

        # Step 3: Render
        scenes_to_process = []
        for i, plan in enumerate([plans_dict[k] for k in sorted(plans_dict)]):
            sn = i + 1
            if plan is None: continue
            if specific_scenes and sn not in specific_scenes: continue
            succ = os.path.join(self.output_dir, fp, f'scene{sn}', 'succ_rendered.txt')
            if os.path.exists(succ):
                print(f'✅ Scene {sn}: already rendered')
            else:
                scenes_to_process.append((i, plan))
                print(f'🔄 Scene {sn}: queued')

        if scenes_to_process:
            print(f'🎬 Rendering {len(scenes_to_process)} scene(s)...')
            tasks = []
            for i, plan in scenes_to_process:
                sn       = i + 1
                sub_dir  = os.path.join(self.output_dir, fp, f'scene{sn}', 'subplans')
                os.makedirs(sub_dir, exist_ok=True)
                trace_f  = os.path.join(sub_dir, 'scene_trace_id.txt')
                trace_id = open(trace_f).read().strip() if os.path.exists(trace_f) else str(uuid.uuid4())
                if not os.path.exists(trace_f): open(trace_f, 'w').write(trace_id)
                tasks.append(self.process_scene(i, scene_outline, plan, topic,
                                                description, max_retries, fp, session_id, trace_id))
            await asyncio.gather(*tasks)

        # Step 4: Combine
        print('🎞️ Combining videos...')
        self.combine_videos(topic)
        print(f'🎉 Pipeline complete for: {topic}')

print('✅ VideoGenerator class ready')

✅ VideoGenerator class ready


In [55]:
# ============================================================
# CELL 10: Full Groq + Manim + Audio Pipeline
# ============================================================

def generate_video_with_groq(topic, max_attempts=3):
    print(f"\n{'='*55}\n🎓 Topic: {topic}\n{'='*55}\n")

    profile             = detect_topic_profile(topic)
    visual_instructions = profile['visual_instructions']
    formula_hints       = get_relevant_formulas(profile)
    graph_instructions  = profile['graph_instructions']

    print('🎙️ Generating narration...')
    script = generate_narration_script(topic)
    print(f'\n📝 Script:\n{"-"*40}\n{script}\n{"-"*40}\n')

    audio_path, audio_duration = text_to_speech(script)
    display(Audio(audio_path))

    error_log  = ''
    video_path = None

    for attempt in range(1, max_attempts + 1):
        print(f'\n🚀 Attempt {attempt}/{max_attempts} (~{audio_duration:.0f}s)...')
        code = generate_manim_code_groq(
            topic, audio_duration,
            visual_instructions, formula_hints,
            graph_instructions, error_log
        )
        print('🎬 Rendering with Manim...')
        success, video_path, error_log = render_manim(code)
        if success:
            print('✅ Rendered successfully!')
            break
        else:
            print(f'❌ Attempt {attempt} failed — retrying with fix...')

    if video_path:
        final = merge_video_audio(video_path, audio_path)
        print('\n🎉 Final video with voiceover:')
        display(Video(final, embed=True, width=720))
    else:
        print(f'\n❌ All {max_attempts} attempts failed.')
        print('Last generated code:')
        with open('main.py') as f: print(f.read())

print('✅ Full pipeline ready')

✅ Full pipeline ready


In [56]:
# Save the code to the right place and render it
code = '''
from manim import *

class GeneratedVideo(Scene):
    def construct(self):
        title = Text("KNN").to_edge(UP)
        self.play(FadeIn(title))

        axes = Axes(
            x_range=[-1, 11, 2],
            y_range=[-1, 11, 2],
            x_length=10,
            y_length=6,
            axis_config={"include_tip": False}
        )
        axes.move_to(ORIGIN)

        dot1 = Dot(axes.coords_to_point(2, 7), color=BLUE)
        dot2 = Dot(axes.coords_to_point(4, 9), color=BLUE)
        dot3 = Dot(axes.coords_to_point(3, 5), color=BLUE)
        dot_new = Dot(axes.coords_to_point(6, 6), color=RED)

        self.play(FadeIn(axes, dot1, dot2, dot3, dot_new))
        self.wait(2)
        self.play(FadeOut(*self.mobjects))

        title = Text("KNN").to_edge(UP)
        self.play(FadeIn(title))

        bc = BarChart(
            values=[5, 6, 3],
            bar_names=["Class A", "Class B", "Class C"],
            y_range=[0, 10, 2],
            bar_width=0.5,
        )
        bc.move_to(ORIGIN)
        self.play(FadeIn(bc))
        self.wait(2)
        self.play(FadeOut(*self.mobjects))

        title = Text("Distance Formula").to_edge(UP)
        self.play(FadeIn(title))
        formula = MathTex(r"d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}").move_to(ORIGIN)
        self.play(FadeIn(formula))
        self.wait(3)
        self.play(FadeOut(*self.mobjects))
'''

with open('/content/scene.py', 'w') as f:
    f.write(code)

import subprocess
result = subprocess.run(
    ["manim", "-ql", "/content/scene.py", "GeneratedVideo"],
    capture_output=True, text=True
)
print("STDOUT:", result.stdout[-2000:])
print("STDERR:", result.stderr[-2000:])

<>:44: SyntaxWarning: invalid escape sequence '\s'
<>:44: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-2536301548.py:44: SyntaxWarning: invalid escape sequence '\s'
  formula = MathTex(r"d = \sqrt{(x_2-x_1)^2 + (y_2-y_1)^2}").move_to(ORIGIN)


STDOUT:     Animation 8 : Using cached     cairo_renderer.py:94
                             data (hash :                                       
                             4072820271_230903733_223132457                     
                             )                                                  
[02/19/26 20:11:21] INFO     Animation 9 : Using cached     cairo_renderer.py:94
                             data (hash :                                       
                             4072820271_565586794_577245184                     
                             )                                                  
                    INFO     Animation 10 : Using cached    cairo_renderer.py:94
                             data (hash :                                       
                             4072820271_4277570474_16697313                     
                             09)                                                
                    INFO     Animation 11 : U

In [57]:
print(_prompt_manim_cheatsheet)

Use Scene, self.play(), self.wait(), Text(), MathTex(), FadeIn/Out()


In [58]:
import inspect
from task_generator import prompts_raw
print(inspect.getfile(prompts_raw))

/content/task_generator/prompts_raw.py


In [63]:
# ============================================================
# CELL 11: ▶️ RUN — Enter your topic here
# ============================================================
topic = input('🎓 Enter topic (e.g. Pythagorean Theorem): ')
generate_video_with_groq(topic, max_attempts=3)

🎓 Enter topic (e.g. Pythagorean Theorem): dijiskti algorathim

🎓 Topic: dijiskti algorathim

🎨 Using general visual style
🎙️ Generating narration...

📝 Script:
----------------------------------------
Imagine you're trying to find the shortest path to a new restaurant in a busy city, with thousands of possible routes to choose from, how would you determine the most efficient one, a dijkstra algorithm is a powerful tool that can help you find the shortest path between two points in a complex network, it was first proposed by edsger dijkstra in 1959, the algorithm works by assigning a weight or cost to each connection between points, and then using a step-by-step process to explore all the possible paths, it's like a puzzle, where the algorithm systematically evaluates each piece to find the optimal solution, as it navigates through the network, it keeps track of the shortest distance from the starting point to every other point, and once it reaches the destination, it can reconstruct th


🚀 Attempt 1/3 (~78s)...
🎬 Rendering with Manim...
✅ Rendered successfully!
🎞️ Final video: output/final_video.mp4

🎉 Final video with voiceover:
